
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

In [ ]:
# Imports api
# ----------------------------------------------------------
import requests
import pandas as pd
import os
from tqdm import tqdm
from src import soporte_extraccion as se
import dotenv
dotenv.load_dotenv()

# Imports web scraping
# ----------------------------------------------------------
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium import webdriver
import os
import re
import zipfile
import shutil

# Extracción de datos de la API de la REE

### REE - Demanda electrica

In [ ]:

host = os.getenv("host_ree")
headers={"Accept": "application/json",
"Content-Type": "application/json",
"Host": host}

# Creamos la carpeta destino en caso de no estar creada
ruta_ree_demanda = os.getenv("ruta_ree_demanda").format()
os.makedirs(ruta_ree_demanda, exist_ok=True)

anios = ["2019", "2020", "2021"]
se.extraer_datos_ree_demanda(ruta_ree_demanda, headers, anios)

### REE - Generación electrica

In [ ]:
host = os.getenv("host_ree")
headers={"Accept": "application/json",
"Content-Type": "application/json",
"Host": host}

# Creamos la carpeta destino en caso de no estar creada
ruta_ree_generacion = os.getenv("ruta_ree_generacion").format()
os.makedirs(ruta_ree_generacion, exist_ok=True)

anios = ["2019", "2020", "2021"]
se.extraer_datos_ree_generacion(ruta_ree_generacion, headers, anios)

# Extracción de datos con Web Scraping del INE

### INE - Datos demográficos

In [ ]:
url_demograficos = os.getenv("url_ine_demograficos")

ruta_ine_demograficos = os.getenv("ruta_ine_demograficos").format()

prefs = {
    "download.default_directory": ruta_ine_demograficos,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
}

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", prefs)

se.extraer_datos_ine_demograficos(url_demograficos, chrome_options)

### INE - Datos económicos

In [ ]:
url_economicos = os.getenv("url_ine_economicos")

ruta_ine_economicos = os.getenv("ruta_ine_economicos").format()

prefs = {
    "download.default_directory": ruta_ine_economicos,
    "download.prompt_for_download": False,
    "directory_upgrade": True,
}

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", prefs)

se.extraer_datos_ine_economicos(url_economicos, chrome_options)